In [470]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

import string
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from pprint import pprint
from unidecode import unidecode
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
import warnings
from tqdm._tqdm_notebook import tqdm_notebook
from scipy.spatial import distance
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
import tensorflow as tf
import transformers
from sent2vec.vectorizer import Vectorizer as S2vectorizer
tqdm_notebook.pandas()
warnings.filterwarnings("ignore")

#import pipelines
from transformers import pipeline
nlp2 = pipeline("question-answering")

pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

In [459]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [695]:
query = "Broad terms in the laws allow for what?"
query = re.sub(r'[^\w\s]', '', query)
result = next(parser.raw_parse((query).lower()))
query_split = query.split(" ")
new_query = ''

if query_split[0].lower() in ['what', 'when', 'why', 'how', 'where', 'which', 'who']:
    for subtree in result[0][1]:
    #     print(subtree)
        if subtree.label() == 'NP' or subtree.label() == 'N'or subtree.label() == 'VP' or subtree.label() == 'WHNP':
    #         print(subtree.leaves())
            new_query += ' '.join(subtree.leaves()) + " "
    #         print(new_query)
else:
    for subtree in result:
    #     print(subtree)
        if subtree.label() == 'NP' or subtree.label() == 'N'or subtree.label() == 'VP' or subtree.label() == 'WHNP':
    #         print(subtree.leaves())
            new_query += ' '.join(subtree.leaves()) + " "

if len(new_query) == 0:
    new_query += query.lower()
    
print(new_query)

broad terms in the laws allow for what


In [696]:
context ="There are 17 laws in the official Laws of the Game, each containing a collection of stipulation and guidelines. The same laws are designed to apply to all levels of football, although certain modifications for groups such as juniors, seniors, women and people with physical disabilities are permitted. The laws are often framed in broad terms, which allow flexibility in their application depending on the nature of the game. The Laws of the Game are published by FIFA, but are maintained by the International Football Association Board (IFAB). In addition to the seventeen laws, numerous IFAB decisions and other directives contribute to the regulation of football."

In [697]:
context = context.lower()
context_split = re.split('[!#$%&\()*+-./:;<=>?@[\\]^_`{|}~]', context)
# '[!#$%&\()*+-./:;<=>?@[\\]^_`{|}~]'
for i in range(len(context_split)):
    context_split[i] = context_split[i].strip()
    
context_split = [x for x in context_split if len(x)>0]

In [698]:
clauses = []
labels = ['SBAQ', 'S', 'SQ', 'SBAR', 'FRAG', 'SBARQ']
for cs in context_split:
    result = next(parser.raw_parse(cs))
    if result[0].label() in labels:
        clauses.append(' '.join(result[0].leaves()))
print(clauses)

['there are 17 laws in the official laws of the game', 'each containing a collection of stipulation and guidelines', 'the same laws are designed to apply to all levels of football', 'although certain modifications for groups such as juniors', 'women and people with physical disabilities are permitted', 'the laws are often framed in broad terms', 'which allow flexibility in their application depending on the nature of the game', 'the laws of the game are published by fifa', 'but are maintained by the international football association board', 'in addition to the seventeen laws', 'numerous ifab decisions and other directives contribute to the regulation of football']


In [701]:
clauses_dictionary = {}
clause_counter = 0
np_dictionary = {}
# labels = ['SBAQ', 'S', 'SQ', 'SBAR', 'FRAG', 'SBARQ']

for clause in clauses:
    result = next(parser.raw_parse(clause))
    if result[0].label() in labels:
        clause_counter += 1
        clauses_dictionary[str(clause_counter)] = {}
        clauses_dictionary[str(clause_counter)][str(clause_counter) + "_" + "clause"] = ' '.join(result[0].leaves())
        np_dictionary[str(clause_counter) + "_NVP_" + "clause"] = ''
        
        inner_clause1 = 0
        
        for subtree in result[0]:
#             print(subtree)
#             print("===")
            if subtree.label() in labels:
                inner_clause1 += 1
                clauses_dictionary[str(clause_counter)][str(clause_counter) + "_" + str(inner_clause1) + "_" + "clause"] = ' '.join(subtree.leaves())

                inner_clause2 = 0
                for subtree2 in subtree:
                    continue
                    
            elif subtree.label() == 'NP' or subtree.label() == 'VP' or subtree.label() == 'PP' or subtree.label() == 'S':
#                 print("***")
#                 print(subtree)
#                 print("***")
                np_dictionary[str(clause_counter) + "_NVP_" + "clause"] += ' '.join(subtree.leaves()) + " "
#                 break

print("Original question:", query)
question_np = new_query
print("Noun phrase after POS tagging of question:", question_np)
question_vector = text_to_vector(question_np)
cosine_np_dict = {}
cosine_np_dict_lemm = {}

question_np_lemm = text = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in question_np.split() if word not in stop_words])
question_vector_lemm = text_to_vector(question_np_lemm)

for np in np_dictionary:
    np_vector = text_to_vector(np_dictionary[np])
    cosine_np_dict[np] = get_cosine(question_vector, np_vector)
    
    np_lemm = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in np_dictionary[np].split() if word not in stop_words])
    np_vector_lemm = text_to_vector(np_lemm)
    cosine_np_dict_lemm[np] = get_cosine(question_vector_lemm, np_vector_lemm)
    
# print(np_dictionary)
import operator
max_clause_id, max_sim = max(cosine_np_dict.items(), key=operator.itemgetter(1))[0], max(cosine_np_dict.items(), key=operator.itemgetter(1))[1]

if max_sim >= 0.2:
    clause_id = max_clause_id[:max_clause_id.find("NVP")-1]

    answer = clauses_dictionary[clause_id][clause_id + "_clause"]
    answer_split = answer.split(" ")
    if answer_split[0] in ["and", 'but', 'or', 'nor', 'after', 'before', 'although']:
        answer_split.pop(0)
    answer_return = " ".join(answer_split)
    print("Answer:", answer_return)
else:
    print("Unable to retrieve answer")


# max_clause_id_lemm, max_sim_lemm = max(cosine_np_dict_lemm.items(), key=operator.itemgetter(1))[0], max(cosine_np_dict_lemm.items(), key=operator.itemgetter(1))[1]

# if max_sim_lemm >= 0.25:
#     clause_id_lemm = max_clause_id_lemm[:max_clause_id_lemm.find("NVP")-1]

#     answer_lemm = clauses_dictionary[clause_id_lemm][clause_id_lemm + "_clause"]
#     answer_split_lemm = answer_lemm.split(" ")
#     if answer_split_lemm[0] in ["and", 'but', 'or', 'nor', 'after', 'before', 'although']:
#         answer_split_lemm.pop(0)
#     answer_return_lemm = " ".join(answer_split_lemm)
#     print("Answer:", answer_return_lemm)
# else:
#     print("Unable to retrieve answer")

# print(np_dictionary)
# print(cosine_np_dict)
# print(cosine_np_dict_lemm)

Original question: Broad terms in the laws allow for what
Noun phrase after POS tagging of question: broad terms in the laws allow for what
Answer: the laws are often framed in broad terms
Answer: the laws are often framed in broad terms


# IGNORE EVERYTHING FROM HERE ONWARDS

In [267]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


text1 = "the head of state of delhi"
text2 = "the official title of delhi head of state"
text3 = "the law enacted by the parliament"

vector1 = text_to_vector(text1)
vector2 = text_to_vector(text2)
vector3 = text_to_vector(text3)

cosine12 = get_cosine(vector1, vector2)
cosine23 = get_cosine(vector2, vector3)

print("Cosine12:", cosine12)
print("Cosine23:", cosine23)

Cosine12: 0.8944271909999159
Cosine23: 0.22360679774997896


In [56]:
stop_words = stopwords.words("english")

def clean_lowercase_nostop(text):
    text = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
#     text = ' '.join([ps.stem(word) for word in text.split() if word not in stop_words])
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text.lower()

def generate_ners(text):
    result = ' '.join([unidecode(X.text.lower()) for X in nlp(text).ents])
    return result

In [65]:
df2 = pd.read_csv('legal_squad_data.csv', encoding='utf-8').loc[:, ['context', 'question', 'answer']]
df = df2.drop_duplicates(subset=['context', 'question'])

df['context'] = df['context'].apply(lambda x: str(x).lower())
df['context_ner'] = df['context'].progress_apply(generate_ners)
df['cleaned_lowercase_nostop'] = df['context'].progress_apply(clean_lowercase_nostop)

In [66]:
df['combined'] = df['context_ner'].map(str) + " " + df['cleaned_lowercase_nostop'].map(str)

In [330]:
df.sample(3)

,context,question,answer,context_ner,cleaned_lowercase_nostop,combined
696,"a treaty is null and void if it is in violation of a peremptory norm. these norms, unlike other principles of customary law, are recognized as permitting no violations and so cannot be altered through treaty obligations. these are limited to such universally accepted prohibitions as those against the aggressive use of force, genocide and other crimes against humanity, piracy, hostilities directed at civilian population, racial discrimination and apartheid, slavery and torture, meaning that no state can legally assume an obligation to commit or permit such acts.",What is an example of a type of universally prohibited action that no state can legally assume an obligation to commit or permit through a treaty?,genocide,,treaty null void violation peremptory norm norms unlike principle customary law recognized permitting violation cannot altered treaty obligations limited universally accepted prohibition aggressive use force genocide crime humanity piracy hostility directed civilian population racial discrimination apartheid slavery torture meaning state legally assume obligation commit permit acts,treaty null void violation peremptory norm norms unlike principle customary law recognized permitting violation cannot altered treaty obligations limited universally accepted prohibition aggressive use force genocide crime humanity piracy hostility directed civilian population racial discrimination apartheid slavery torture meaning state legally assume obligation commit permit acts
2145,"the unicameral legislature (the fono) consists of 49 members serving 5-year terms. forty-seven are matai title-holders elected from territorial districts by samoans; the other two are chosen by non-samoans with no chiefly affiliation on separate electoral rolls. universal suffrage was adopted in 1990, but only chiefs (matai) may stand for election to the samoan seats. there are more than 25,000 matais in the country, about 5% of whom are women. the prime minister, chosen by a majority in the fono, is appointed by the head of state to form a government. the prime minister's choices for the 12 cabinet positions are appointed by the head of state, subject to the continuing confidence of the fono.",What percentage of female matais are there in Samoa?,5%,"49 forty-seven two universal suffrage 1990 more than 25,000 about 5% 12",unicameral legislature the fono consists 49 member serving 5 year terms forty seven matai title holder elected territorial district samoans two chosen non samoans chiefly affiliation separate electoral rolls universal suffrage adopted 1990 chief matai may stand election samoan seats 25 000 matai country 5 women prime minister chosen majority fono appointed head state form government prime minister s choice 12 cabinet position appointed head state subject continuing confidence fono,"49 forty-seven two universal suffrage 1990 more than 25,000 about 5% 12 unicameral legislature the fono consists 49 member serving 5 year terms forty seven matai title holder elected territorial district samoans two chosen non samoans chiefly affiliation separate electoral rolls universal suffrage adopted 1990 chief matai may stand election samoan seats 25 000 matai country 5 women prime minister chosen majority fono appointed head state form government prime minister s choice 12 cabinet position appointed head state subject continuing confidence fono"
4103,"there are 17 laws in the official laws of the game, each containing a collection of stipulation and guidelines. the same laws are designed to apply to all levels of football, although certain modifications for groups such as juniors, seniors, women and people with physical disabilities are permitted. the laws are often framed in broad terms, which allow flexibility in their application depending on the nature of the game. the laws of the game are published by fifa, but are maintained by the international football association board

In [257]:
# Getting trigrams  
bigram_vectorizer = CountVectorizer(ngram_range = (1,2)) 
bigram_X1 = bigram_vectorizer.fit_transform(df['cleaned_lowercase_nostop'])  
bigram_features = (bigram_vectorizer.get_feature_names()) 
  
# Applying TFIDF 
bigram_vectorizer = TfidfVectorizer(ngram_range = (1,2)) 
bigram_tfidf = bigram_vectorizer.fit_transform(df['cleaned_lowercase_nostop']) 
bigram_scores = (bigram_tfidf.toarray())

In [261]:
def get_similar_doc_bigram(tfidfvectorizer, docs_tfidf, query):
    query_tfidf = tfidfvectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    cosineSimilarities = np.array(cosineSimilarities)
    return cosineSimilarities.argsort()[-5:][::-1]

def get_bigram_matching_docs(for_bigram_matching):
#     for_bigram_matching = re.sub('[%s]' % re.escape(string.punctuation), '', query)
#     for_bigram_matching = re.sub('\s{2,}', " ", for_bigram_matching).lower()
#     for_bigram_matching = ' '.join([word for word in for_bigram_matching.split() if word not in stop_words])

    bigram_matching_ids = get_similar_doc_bigram(bigram_vectorizer, bigram_tfidf, for_bigram_matching)
    
    return df.iloc[bigram_matching_ids, df.columns.get_loc('context')]

def get_bigram_answer(query):
    for_bigram_matching = re.sub('[%s]' % re.escape(string.punctuation), '', query)
    for_bigram_matching = re.sub('\s{2,}', " ", for_bigram_matching).lower()
    for_bigram_matching = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in for_bigram_matching.split() if word not in stop_words])

    print("Question:", query)
    print("Processed question:", for_bigram_matching)
    print()
    
    contexts = get_bigram_matching_docs(for_bigram_matching)
    
    counter = 1
    for context in contexts:
        print("Context", counter, ":")
        print(context)
        print()
        counter += 1

get_bigram_answer("what is criminal law in united states?")

Question: what is criminal law in united states?
Processed question: criminal law united state

[0.10918998 0.10918998 0.10918998 ... 0.00680466 0.00680466 0.00680466]


AttributeError: 'str' object has no attribute 'array'

In [77]:
result = next(parser.raw_parse(context))
#     print(result[0])
print(result[0])
print("==========")
print(result[0][0])
# for s in result:
#     for subtree in s.subtrees():
#         if subtree.label() == 'NP':
#             len(subtree.leaves()) < 10:
#             print(subtree.leaves())
#             print("====")
#         if subtree.label() in labels:
#     if s.label() in labels:
#         print(s.leaves())

(S
  (S
    (NP
      (NP (DT the) (NN head))
      (PP (IN of) (NP (NP (NN state)) (PP (IN of) (NP (NNP delhi))))))
    (VP
      (VBZ is)
      (NP
        (NP (DT the) (NN lieutenant) (NN governor))
        (PP
          (IN of)
          (NP
            (NP
              (NP (DT the) (NN union) (NN territory))
              (PP (IN of) (NP (NNP delhi))))
            (, ,)
            (VP
              (VBN appointed)
              (PP
                (IN by)
                (NP
                  (NP (DT the) (NN president))
                  (PP (IN of) (NP (NN india)))))
              (PP
                (IN on)
                (NP
                  (NP (DT the) (NN advice))
                  (PP
                    (IN of)
                    (NP (DT the) (JJ central) (NN government)))))))))))
  (CC and)
  (S
    (NP (DT the) (NN post))
    (VP
      (VBZ is)
      (ADJP (RB largely) (JJ ceremonial))
      (, ,)
      (SBAR
        (IN as)
        (S
          (NP
            (NP

In [69]:
query = get_bigram_answer("What is the official title of Delhi's head of state?")

Question: What is the official title of Delhi's head of state?
Processed question: official title delhi head state

Context 1 :
marshall's words and future court decisions gave congress much latitude in delegating powers. it was not until the 1930s that the supreme court held a delegation of authority unconstitutional. in a case involving the creation of the national recovery administration called a.l.a. schechter poultry corp. v. united states, 295 u.s. 495 (1935), congress could not authorize the president to formulate codes of "fair competition." it was held that congress must set some standards governing the actions of executive officers. the court, however, has deemed that phrases such as "just and reasonable," "public interest" and "public convenience" suffice.

(S
  (S
    (NP
      (NP (NP (NNP marshall) (POS 's)) (NNS words))
      (CC and)
      (NP (JJ future) (NN court) (NNS decisions)))
    (VP
      (VBD gave)
      (NP (NN congress))
      (NP
        (NP (JJ much) (NN l

(S
  (S
    (NP
      (NP (NP (NNP marshall) (POS 's)) (NNS words))
      (CC and)
      (NP (JJ future) (NN court) (NNS decisions)))
    (VP
      (VBD gave)
      (NP (NN congress))
      (NP
        (NP (JJ much) (NN latitude))
        (PP (IN in) (S (VP (VBG delegating) (NP (NNS powers))))))))
  (. .)
  (S
    (NP (PRP it))
    (VP
      (VBD was)
      (RB not)
      (SBAR
        (IN until)
        (S
          (NP
            (NP (DT the) (NNS 1930s))
            (SBAR
              (WHNP (WDT that))
              (S
                (VP
                  (FRAG
                    (NP
                      (NP (DT the) (JJ supreme) (NN court))
                      (SBAR
                        (S
                          (VP
                            (VBD held)
                            (S
                              (NP
                                (NP (DT a) (NN delegation))
                                (PP (IN of) (NP (NN authority))))
                           

In [6]:
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load()

In [13]:
def print_fine_pos(token):
    return (token.tag_)

def pos_tags(sentence):
    tokens = nlp(sentence)
    tags = []
    for tok in tokens:
        tags.append((tok,print_fine_pos(tok)))
    return tags

print(pos_tags(a))

[(we, 'PRP'), (had, 'VBD'), (crispy, 'NNP'), (dosa, 'NNP'), (and, 'CC'), (it, 'PRP'), (was, 'VBD'), (very, 'RB'), (nice, 'JJ'), (., '.'), (i, 'PRP'), (like, 'VBP'), (dosa, 'NNP'), (., '.')]


In [23]:
import os
os.environ['JAVAHOME'] = 'D:/Java/bin'
os.environ['STANFORD_PARSER'] = '/Applications/stanford-parser'
os.environ['STANFORD_MODELS'] = '/Applications/stanford-parser'

In [30]:
from nltk.parse.corenlp import CoreNLPParser
parser = CoreNLPParser(url='http://localhost:9000')
labels = ['S', 'SBAR', 'FRAG', 'SQ']
# sent = 'room was smelly and the toilet was nice. the staffs were rude.'

a = "we had crispy dosa and it was very nice. i like dosa"
sentences = a.split(".")

for sentence in sentences:
    result = next(parser.raw_parse(sentence))
#     print(result[0])
    for s in result:
        print(s)
#     for subtree in result.subtrees():
#         if subtree.label() in labels:
        if s.label() in labels:
            print(s.leaves())
    print("=============================")

(S
  (S (NP (PRP we)) (VP (VBD had) (NP (JJ crispy) (NN dosa))))
  (CC and)
  (S (NP (PRP it)) (VP (VBD was) (ADJP (RB very) (JJ nice)))))
['we', 'had', 'crispy', 'dosa', 'and', 'it', 'was', 'very', 'nice']
(S (NP (PRP i)) (VP (VBP like) (NP (NN dosa))))
['i', 'like', 'dosa']
